In [1]:
import pandas as pd
import numpy as np
import re
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt

In [46]:
tables=camelot.read_pdf("axis03.pdf",flavor='stream' ,pages = "all",edge_tol=500)            
if len(tables) !=0 :
    df=pd.DataFrame();    tmp=pd.DataFrame(); 
    for i in range(len(tables)) :
        tmp=tables[i].df  
        df=pd.concat([df,tmp]).drop_duplicates(keep='first').reset_index(drop=True)

In [47]:
# df.to_csv("error1.csv",index=0)

In [48]:
tmp=tables[2].df
tmp

,0,1,2,3,4,5
0,Tran Date\nValue Date\nTransaction Particulars,Chq No.,Amount,DR|CR,Balance,Branch Name
1,OPENING BALANCE :,,,,25184.55,
2,,,,,,ATM
3,14-01-2019\n14-01-2019\nEDC/00053-00053-/M0370...,,1001.90,CR,49324.09,RECONCILATION
4,,,,,,CENTRE
5,,,,,,ZAMROODPUR
6,14-01-2019\n14-01-2019\nIMPS/P2A/901412924696/...,,497.00,CR,49821.09,
7,,,,,,DEL DL
8,,,,,,ZAMROODPUR
9,14-01-2019\n14-01-2019\nIMPS/P2A/901412927199/...,,400.00,CR,50221.09,


In [49]:
idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
print("Columns:",tmp.columns)

Columns: Index(['Tran Date\nValue Date\nTransaction Particulars', 'Chq No.', 'Amount',
       'DR|CR', 'Balance', 'Branch Name'],
      dtype='object', name=0)


In [50]:
tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
tmp=pd.DataFrame(tmp.T.drop_duplicates().T)
# tmp.drop(tmp.nunique(dropna=False)[(tmp.nunique(dropna=False) == 1)].index, axis=1,inplace=True)

In [51]:
tmp.columns = ["A","B","C","D","E","F"]

In [52]:
tmp.reset_index(drop = True, inplace=True)

In [53]:
tmp.head()

,A,B,C,D,E,F
0,,,,,,ATM
1,14-01-2019\n14-01-2019\nEDC/00053-00053-/M0370...,,1001.90,CR,49324.09,RECONCILATION
2,,,,,,CENTRE
3,,,,,,ZAMROODPUR
4,14-01-2019\n14-01-2019\nIMPS/P2A/901412924696/...,,497.00,CR,49821.09,


In [54]:
tmp.to_csv("error1.csv",index=0)

In [55]:
tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])

In [56]:
tmp1

,Z1,Z2,Z3
0,,None,None
1,14-01-2019,14-01-2019,EDC/00053-00053-/M037011032984700
2,,None,None
3,,None,None
4,14-01-2019,14-01-2019,IMPS/P2A/901412924696/919971624523/INETIMPS00073
5,,None,None
6,,None,None
7,14-01-2019,14-01-2019,IMPS/P2A/901412927199/919971624523/MOBUA01304940
8,,None,None
9,,None,None


In [57]:
# tmp1.replace(r'^\s*$', np.nan, regex=True, inplace=True)
# tmp1.fillna(value=pd.np.nan, inplace=True)



# match = re.search(r'(\d+-\d+-\d+)',i)
# match.group(1)

In [58]:
# tmp1["flag"] = tmp1["Z2"].astype(str)+tmp1["Z3"].astype(str)

In [59]:
tmp1.to_csv("tmp1.csv", index=0)

In [60]:
# frames = [tmp1, tmp]
# result = pd.concat(frames)

result = pd.concat([tmp1, tmp], axis=1, sort=False)

In [62]:
result.to_csv("result.csv",index=0)

In [63]:
result.drop('A' ,axis=1, inplace=True)

In [64]:
result.columns

Index(['Z1', 'Z2', 'Z3', 'B', 'C', 'D', 'E', 'F'], dtype='object')

In [65]:
result["xx"] = result["Z1"]

In [66]:
result.to_csv("result.csv",index=0)

In [67]:
for i,j in enumerate(result["Z1"]):
    if re.search(r'(\d+-\d+-\d+)',j):
        result.at[i, 'xx'] = ""
        pass
    elif re.search(r'^\s*$',j):
        result.at[i, 'xx'] = ""
        pass
    else:
        print(j) 

BY CASH DEPOSIT-
BNA/DPRH329801/6971/170119/SOUTH D
BY CASH DEPOSIT-
BNA/DPRH329801/8113/230119/SOUTH D
BY CASH DEPOSIT-
BNA/DPRH329801/8115/230119/SOUTH D
BY CASH DEPOSIT-
BNA/DPRH329801/8913/280119/SOUTH D


In [68]:
result

,Z1,Z2,Z3,B,C,D,E,F,xx
0,,None,None,,,,,ATM,
1,14-01-2019,14-01-2019,EDC/00053-00053-/M037011032984700,,1001.90,CR,49324.09,RECONCILATION,
2,,None,None,,,,,CENTRE,
3,,None,None,,,,,ZAMROODPUR,
4,14-01-2019,14-01-2019,IMPS/P2A/901412924696/919971624523/INETIMPS00073,,497.00,CR,49821.09,,
5,,None,None,,,,,DEL DL,
6,,None,None,,,,,ZAMROODPUR,
7,14-01-2019,14-01-2019,IMPS/P2A/901412927199/919971624523/MOBUA01304940,,400.00,CR,50221.09,,
8,,None,None,,,,,DEL DL,
9,,None,None,,,,,ZAMROODPUR,


In [69]:
result.fillna(value=pd.np.nan, inplace=True)

In [73]:
# for i in result["Z3"]:
#     print(type(i))

In [82]:
result['X'] = result['Z3'].fillna('') + result['xx'].fillna('')

In [83]:
result.to_csv("result.csv",index=0)